In [1]:
import numpy as np
import src.realtime_utils as utils
import torch
from pylsl import StreamInlet, resolve_stream

ModuleNotFoundError: No module named 'typing_extensions'

In [ ]:
# INIT
filt_ord = 2
freq_limits = np.asarray([[1,100]]) 
freq_limits_names = ['1_100Hz']
sample_duration = 125
sampling_frequency = 250
subject = FLAGS.subjects[0]
electrode_names =  ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
filters = utils.init_filters(freq_limits, sampling_frequency, filt_type = 'bandpass', order=filt_ord)
segments, labels, predictions = [], [], []
total_outlier = 0

# init DL model
net = utils.EEGNET()
net.load_state_dict(torch.load(f'finetuned_models/{subject}/EEGNET_trialnum4'))
net = net.float()
net.eval()

In [ ]:
# below code is for initializing the streaming layer which will help us capture data later
finished = False
streams = resolve_stream()
inlet = StreamInlet(streams[0])
sig_tot = ''
i = 0

In [2]:
columns=['Time','FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8','AccX','AccY','AccZ','Gyro1','Gyro2','Gyro3',
                                  'Battery','Counter','Validation']

data_dict = dict((k, []) for k in columns)

#updating database with captured data from EEG
def update_data(data,res):
    i = 0
    for key in list(data.keys()):
        data[key].append(res[i])
        i = i +1
    return data

In [3]:
while not finished:
    # capturing sample first using inlet.pull_sample()
    sample, timestamp = inlet.pull_sample()
    print( timestamp, sample)

In [ ]:
#     res = [timestamp] + sample
#     data_dict = update_data(data_dict,res)
#     times.append(timestamp)

# data_dict['Class'] = classes
# # making dictionary into a dataframe for saving it as csv
# record_data = pd.DataFrame.from_dict(data_dict)